In [3]:
%run dog-breed-cnn.py

# Load train, test and validation datasets
train_files, train_targets = load_dataset('Data/dogImages/train')
valid_files, valid_targets = load_dataset('Data/dogImages/valid')
test_files, test_targets = load_dataset('Data/dogImages/test')

Using TensorFlow backend.
